In [1]:
# notebooks/demo.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Skolyoz Tespiti Demo\n",
    "\n",
    "Bu notebook, skolyoz tespiti ve Cobb açısı hesaplama projesinin temel kullanımını gösterir."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Kurulum ve İçe Aktarma\n",
    "\n",
    "Gerekli modülleri içe aktaralım:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import os\n",
    "import sys\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import cv2\n",
    "\n",
    "# Proje kök dizinini ekle\n",
    "sys.path.append('..')\n",
    "\n",
    "# Proje modüllerini içe aktar\n",
    "from src.data.loader import XRayLoader\n",
    "from src.data.preprocessing import XRayPreprocessor\n",
    "from src.models.unet import SpineSegmentationModel\n",
    "from src.spine.cobb_angle import CobbAngleCalculator\n",
    "from src.visualization.visualize import ResultVisualizer\n",
    "from src.main import SkolyozDetector\n",
    "from src.utils.helpers import print_system_info, check_gpu_availability"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Sistem Bilgilerini Kontrol Et"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Sistem bilgilerini göster\n",
    "system_info = print_system_info()\n",
    "\n",
    "# GPU kullanılabilirliğini kontrol et\n",
    "gpu_available, device_name = check_gpu_availability()\n",
    "print(f\"GPU kullanılabilir: {gpu_available}, Cihaz: {device_name}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Tek Bir Görüntü İşleme\n",
    "\n",
    "Skolyoz dedektörünü başlatıp tek bir röntgen görüntüsünü işleyelim:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Skolyoz dedektörünü oluştur\n",
    "detector = SkolyozDetector()\n",
    "\n",
    "# Örnek bir görüntü yolu\n",
    "image_path = \"../data/raw/sample_xray.jpg\"  # Kendi dosya yolunuzu belirtin\n",
    "\n",
    "# Görüntünün var olduğunu kontrol et\n",
    "if not os.path.exists(image_path):\n",
    "    print(f\"HATA: Görüntü bulunamadı: {image_path}\")\n",
    "    print(\"Lütfen geçerli bir X-ray görüntüsü ekleyin veya yol belirtin.\")\n",
    "else:\n",
    "    # Görüntüyü interaktif olarak işle (sonuçları göster)\n",
    "    result = detector.interactive_process(image_path)\n",
    "    \n",
    "    if result.get('success', False):\n",
    "        print(f\"\\nSonuçlar:\")\n",
    "        print(f\"  Cobb Açısı: {result.get('cobb_angle', float('nan')):.1f}°\")\n",
    "        print(f\"  İşlem Süresi: {result.get('processing_time', 0):.2f} saniye\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Adım Adım İşlem\n",
    "\n",
    "Şimdi skolyoz tespiti sürecinin her adımını ayrı ayrı inceleyelim:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Yeni bir görüntü seçin (veya aynısını kullanın)\n",
    "image_path = \"../data/raw/sample_xray.jpg\"  # Kendi dosya yolunuzu belirtin\n",
    "\n",
    "if not os.path.exists(image_path):\n",
    "    print(f\"HATA: Görüntü bulunamadı: {image_path}\")\n",
    "else:\n",
    "    # 1. Görüntü Yükleme\n",
    "    loader = XRayLoader()\n",
    "    image = loader.load_image(image_path)\n",
    "    \n",
    "    # Görüntüyü göster\n",
    "    plt.figure(figsize=(12, 10))\n",
    "    \n",
    "    plt.subplot(2, 3, 1)\n",
    "    plt.imshow(image, cmap='gray')\n",
    "    plt.title(\"1. Orijinal X-ray\")\n",
    "    plt.axis('off')\n",
    "    \n",
    "    # 2. Ön İşleme\n",
    "    preprocessor = XRayPreprocessor()\n",
    "    processed = preprocessor.apply_all(image)\n",
    "    \n",
    "    plt.subplot(2, 3, 2)\n",
    "    plt.imshow(processed['enhanced'], cmap='gray')\n",
    "    plt.title(\"2. İşlenmiş Görüntü\")\n",
    "    plt.axis('off')\n",
    "    \n",
    "    # 3. Omurga Segmentasyonu\n",
    "    model = SpineSegmentationModel()\n",
    "    spine_mask_prob = model.segment_spine(processed['enhanced'])\n",
    "    spine_mask = model.get_binary_mask(spine_mask_prob, threshold=0.5)\n",
    "    \n",
    "    plt.subplot(2, 3, 3)\n",
    "    plt.imshow(spine_mask, cmap='gray')\n",
    "    plt.title(\"3. Omurga Segmentasyonu\")\n",
    "    plt.axis('off')\n",
    "    \n",
    "    # 4. Cobb Açısı Hesaplama\n",
    "    cobb_calculator = CobbAngleCalculator()\n",
    "    cobb_result = cobb_calculator.calculate_angle(spine_mask)\n",
    "    \n",
    "    # 5. Eğri Noktalarını Göster\n",
    "    plt.subplot(2, 3, 4)\n",
    "    plt.imshow(processed['enhanced'], cmap='gray')\n",
    "    \n",
    "    if cobb_result.get('success', False):\n",
    "        curve_points = np.array(cobb_result['curve_points'])\n",
    "        plt.plot(curve_points[:, 0], curve_points[:, 1], 'b-', linewidth=2)\n",
    "        \n",
    "        # Kritik noktaları göster\n",
    "        inflection = cobb_result['inflection_point']\n",
    "        upper = cobb_result['upper_end_vertebra']\n",
    "        lower = cobb_result['lower_end_vertebra']\n",
    "        \n",
    "        plt.plot(inflection[0], inflection[1], 'go', markersize=8)\n",
    "        plt.plot(upper[0], upper[1], 'ro', markersize=8)\n",
    "        plt.plot(lower[0], lower[1], 'ro', markersize=8)\n",
    "        \n",
    "        plt.title(\"4. Omurga Eğrisi ve Kritik Noktalar\")\n",
    "    else:\n",
    "        plt.title(\"4. Omurga Eğrisi Tespit Edilemedi\")\n",
    "    \n",
    "    plt.axis('off')\n",
    "    \n",
    "    # 6. Açı Çizgilerini Göster\n",
    "    plt.subplot(2, 3, 5)\n",
    "    plt.imshow(processed['enhanced'], cmap='gray')\n",
    "    \n",
    "    if cobb_result.get('success', False):\n",
    "        # Cobb açısı hesaplanabildi\n",
    "        angle = cobb_result.get('angle', float('nan'))\n",
    "        \n",
    "        # Açı çizgilerini çiz\n",
    "        upper = np.array(cobb_result['upper_end_vertebra'])\n",
    "        lower = np.array(cobb_result['lower_end_vertebra'])\n",
    "        \n",
    "        upper_slope = cobb_result['upper_slope']\n",
    "        lower_slope = cobb_result['lower_slope']\n",
    "        \n",
    "        # Çizgileri göster\n",
    "        y_len = 50  # Çizgi uzunluğu\n",
    "        \n",
    "        # Üst çizgi\n",
    "        if np.isinf(upper_slope):\n",
    "            x1, y1 = upper[0], upper[1] - y_len\n",
    "            x2, y2 = upper[0], upper[1] + y_len\n",
    "        else:\n",
    "            x1 = upper[0] - y_len * upper_slope\n",
    "            y1 = upper[1] - y_len\n",
    "            x2 = upper[0] + y_len * upper_slope\n",
    "            y2 = upper[1] + y_len\n",
    "        \n",
    "        plt.plot([x1, x2], [y1, y2], 'r-', linewidth=2)\n",
    "        \n",
    "        # Alt çizgi\n",
    "        if np.isinf(lower_slope):\n",
    "            x1, y1 = lower[0], lower[1] - y_len\n",
    "            x2, y2 = lower[0], lower[1] + y_len\n",
    "        else:\n",
    "            x1 = lower[0] - y_len * lower_slope\n",
    "            y1 = lower[1] - y_len\n",
    "            x2 = lower[0] + y_len * lower_slope\n",
    "            y2 = lower[1] + y_len\n",
    "        \n",
    "        plt.plot([x1, x2], [y1, y2], 'r-', linewidth=2)\n",
    "        \n",
    "        plt.title(f\"5. Cobb Açısı: {angle:.1f}°\")\n",
    "    else:\n",
    "        plt.title(\"5. Cobb Açısı Hesaplanamadı\")\n",
    "    \n",
    "    plt.axis('off')\n",
    "    \n",
    "    # 7. Sonuç Görselleştirme\n",
    "    plt.subplot(2, 3, 6)\n",
    "    \n",
    "    # Görselleştiriciyi oluştur\n",
    "    visualizer = ResultVisualizer()\n",
    "    result_image = visualizer.visualize_results(\n",
    "        processed['enhanced'],\n",
    "        spine_mask,\n",
    "        cobb_result,\n",
    "        output_path=None,\n",
    "        show_plot=False\n",
    "    )\n",
    "    \n",
    "    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))\n",
    "    plt.title(\"6. Final Sonuç\")\n",
    "    plt.axis('off')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Sonuçları özetle\n",
    "    if cobb_result.get('success', False):\n",
    "        angle = cobb_result.get('angle', float('nan'))\n",
    "        print(f\"\\nSonuç: Cobb Açısı = {angle:.1f}°\")\n",
    "        \n",
    "        # Açının yorumlanması\n",
    "        if angle < 10:\n",
    "            severity = \"Normal veya minimal eğrilik\"\n",
    "        elif angle < 25:\n",
    "            severity = \"Hafif skolyoz\"\n",
    "        elif angle < 40:\n",
    "            severity = \"Orta şiddetli skolyoz\"\n",
    "        else:\n",
    "            severity = \"Şiddetli skolyoz\"\n",
    "        \n",
    "        print(f\"Değerlendirme: {severity}\")\n",
    "    else:\n",
    "        print(\"\\nSonuç: Cobb açısı hesaplanamadı.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Birden Fazla Görüntü İşleme\n",
    "\n",
    "Bir dizindeki tüm X-ray görüntülerini işleyelim:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# İşlenecek görüntülerin bulunduğu dizin\n",
    "input_dir = \"../data/raw/\"  # Kendi dizin yolunuzu belirtin\n",
    "\n",
    "# Çıktı dizini\n",
    "output_dir = \"../output/demo_results/\"\n",
    "\n",
    "# Dedektör oluştur\n",
    "detector = SkolyozDetector()\n",
    "\n",
    "# Dizindeki tüm görüntüleri işle\n",
    "results = detector.process_directory(\n",
    "    input_dir,\n",
    "    output_dir=output_dir,\n",
    "    save_results=True,\n",
    "    show_results=False,\n",
    "    compare_results=True\n",
    ")\n",
    "\n",
    "# Sonuçları tablo halinde göster\n",
    "if results:\n",
    "    from IPython.display import display, HTML\n",
    "    import pandas as pd\n",
    "    \n",
    "    # Sonuçları DataFrame'e çevir\n",
    "    df = pd.DataFrame([\n",
    "        {\n",
    "            \"Dosya\": os.path.basename(r[\"image_path\"]),\n",
    "            \"Cobb Açısı\": f\"{r.get('cobb_angle', float('nan')):.1f}°\" if r.get('cobb_success', False) else \"N/A\",\n",
    "            \"Başarı\": \"✓\" if r.get('cobb_success', False) else \"✗\",\n",
    "            \"Süre (sn)\": f\"{r.get('processing_time', 0):.2f}\"\n",
    "        } for r in results\n",
    "    ])\n",
    "    \n",
    "    # DataFrame'i göster\n",
    "    display(HTML(df.to_html()))\n",
    "    \n",
    "    # Karşılaştırma görselini göster\n",
    "    comparison_path = os.path.join(output_dir, \"comparison.png\")\n",
    "    if os.path.exists(comparison_path):\n",
    "        plt.figure(figsize=(12, 10))\n",
    "        plt.imshow(plt.imread(comparison_path))\n",
    "        plt.axis('off')\n",
    "        plt.title(\"İşlenen Tüm Görüntülerin Karşılaştırması\")\n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Veri Analizi ve İstatistikler\n",
    "\n",
    "İşlenen tüm görüntülerin sonuçlarını analiz edelim:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "if results:\n",
    "    import pandas as pd\n",
    "    import seaborn as sns\n",
    "    \n",
    "    # Başarılı sonuçları filtrele\n",
    "    successful_results = [r for r in results if r.get('cobb_success', False)]\n",
    "    angles = [r.get('cobb_angle', float('nan')) for r in successful_results]\n",
    "    \n",
    "    if angles:\n",
    "        # İstatistikleri hesapla\n",
    "        angles = np.array(angles)\n",
    "        angles = angles[~np.isnan(angles)]\n",
    "        \n",
    "        if len(angles) > 0:\n",
    "            print(f\"İstatistikler:\")\n",
    "            print(f\"  Ortalama Cobb Açısı: {np.mean(angles):.2f}°\")\n",
    "            print(f\"  Medyan Cobb Açısı: {np.median(angles):.2f}°\")\n",
    "            print(f\"  Minimum Cobb Açısı: {np.min(angles):.2f}°\")\n",
    "            print(f\"  Maksimum Cobb Açısı: {np.max(angles):.2f}°\")\n",
    "            print(f\"  Standart Sapma: {np.std(angles):.2f}°\")\n",
    "            \n",
    "            # Histogram\n",
    "            plt.figure(figsize=(10, 6))\n",
    "            \n",
    "            sns.histplot(angles, bins=10, kde=True)\n",
    "            plt.axvline(np.mean(angles), color='r', linestyle='--', label=f'Ortalama: {np.mean(angles):.2f}°')\n",
    "            plt.axvline(np.median(angles), color='g', linestyle='-.', label=f'Medyan: {np.median(angles):.2f}°')\n",
    "            \n",
    "            plt.title(\"Cobb Açısı Dağılımı\")\n",
    "            plt.xlabel(\"Cobb Açısı (°)\")\n",
    "            plt.ylabel(\"Frekans\")\n",
    "            plt.legend()\n",
    "            plt.grid(True, alpha=0.3)\n",
    "            plt.show()\n",
    "            \n",
    "            # Skolyoz şiddet dağılımı\n",
    "            severity_counts = {\n",
    "                \"Normal (<10°)\": np.sum(angles < 10),\n",
    "                \"Hafif (10-25°)\": np.sum((angles >= 10) & (angles < 25)),\n",
    "                \"Orta (25-40°)\": np.sum((angles >= 25) & (angles < 40)),\n",
    "                \"Şiddetli (≥40°)\": np.sum(angles >= 40)\n",
    "            }\n",
    "            \n",
    "            plt.figure(figsize=(10, 6))\n",
    "            plt.bar(severity_counts.keys(), severity_counts.values(), color=['green', 'blue', 'orange', 'red'])\n",
    "            plt.title(\"Skolyoz Şiddet Dağılımı\")\n",
    "            plt.xlabel(\"Şiddet Kategorisi\")\n",
    "            plt.ylabel(\"Görüntü Sayısı\")\n",
    "            plt.grid(True, alpha=0.3, axis='y')\n",
    "            \n",
    "            for i, v in enumerate(severity_counts.values()):\n",
    "                plt.text(i, v + 0.1, str(v), ha='center')\n",
    "                \n",
    "            plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Özet ve Sonuç\n",
    "\n",
    "Bu demo notebook'unda:\n",
    "\n",
    "1. Skolyoz tespiti için geliştirilen sistemin temel bileşenlerini tanıttık\n",
    "2. Tek bir X-ray görüntüsünün nasıl işlendiğini adım adım gösterdik\n",
    "3. Bir dizindeki tüm görüntüleri toplu işlemeyi öğrendik\n",
    "4. Sonuçları analiz etmek için basit istatistikler ve görselleştirmeler kullandık\n",
    "\n",
    "Daha kapsamlı bilgi için projenin README dosyasına ve kaynak kodlarına bakınız."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


NameError: name 'null' is not defined